In [20]:
!pip3 install pandas


In [1]:
import os
import re
import pandas as pd
import shutil


## Cargar alumnos

Toma los alumnos que estan en alumnos.md y lo carga en un datatable.
El formato del archivo debe ser
```
## Comision N
1. Legajo Apellido, Nombre
```

In [2]:

def cargar_alumos(origen='alumnos.md'):
    # Leer el archivo
    with open(origen, 'r') as file:
        lines = file.readlines()
    
    # Inicializar listas para almacenar los datos
    comisiones = []
    ordenes = []
    legajos = []
    apellidos = []
    nombres = []
    
    
    # Variables para almacenar la comisión actual
    comision_actual = None
    
    # Procesar cada línea del archivo
    for line in lines:
        line = line.strip()
        line = re.sub(r'\|.*', '', line)
        
        if line.startswith('## Comisión'):
            # Extraer el número de la comisión
            comision_actual = int(line.split(' ')[-1])
        elif line and line[0].isdigit():
            # Extraer el número de orden, legajo, apellido y nombre
            match = re.match(r'(\d+)\.\s+(\d+)\s+([^,]+),\s+(.+)(|>.*)?', line)
            if match:
                comisiones.append(comision_actual)
                ordenes.append(int(match.group(1)))
                legajos.append(match.group(2))
                apellidos.append(match.group(3))
                nombres.append(match.group(4))
    
    # Crear el DataFrame
    data = {
        'comision': comisiones,
        'orden': ordenes,
        'legajo': legajos,
        'apellido': apellidos,
        'nombre': nombres
    }
    df = pd.DataFrame(data)
    
    return df

# Uso de la función


## Detectar datos duplicados

Revisa si los legajos estan duplicados (error), y como curiosidad si hay nombre y apellidos repetidos.

In [3]:
def detectar_duplicados(d):
    # Listar registros con legajo duplicado
    legajos_duplicados = d[d['legajo'].duplicated()]
    print("Registros con legajo duplicado:")
    if len(legajos_duplicados) == 0:
        print("No hay legajos duplicados")
    else:
        print("ERROR: Se encontraron legajos duplicados")
        print(legajos_duplicados)

    # Listar registros con nombre duplicado
    nombres_duplicados = d[d['nombre'].duplicated(keep=False)]
    nombres_duplicados.sort_values(by='nombre', inplace=True)
    num_nombres = len(nombres_duplicados['nombre'].unique())

    print(f"\nHay {num_nombres} Nombres duplicados en {len(nombres_duplicados)} registros")
    print("Registros con nombre duplicado:")
    print(nombres_duplicados)

    # Listar registros con apellido duplicado
    apellidos_duplicados = d[d['apellido'].duplicated(keep=False)]
    apellidos_duplicados.sort_values(by='apellido', inplace=True)

    num_apellidos = len(apellidos_duplicados['apellido'].unique())

    print(f"\nHay {num_apellidos} Apellidos duplicados en {len(apellidos_duplicados)} registros")
    print("Registros con apellido duplicado: ")
    print(apellidos_duplicados)

detectar_duplicados(cargar_alumos())

Registros con legajo duplicado:
No hay legajos duplicados

Hay 3 Nombres duplicados en 6 registros
Registros con nombre duplicado:
     comision  orden legajo   apellido        nombre
34          2     35  59450    Skibski       Ignacio
121         7     32  59250   Martinez       Ignacio
36          2     37  59160  Veneziano  Juan Ignacio
139         9     17  59074    Teseyra  Juan Ignacio
84          5     48  59064     Urbani    Juan Pablo
128         9      6  59313   Pisentte    Juan Pablo

Hay 13 Apellidos duplicados en 29 registros
Registros con apellido duplicado: 
     comision  orden legajo     apellido                      nombre
4           2      5  59731      Alvarez                     Nicolás
5           2      6  58828      Alvarez              Nicolás Nahuel
6           2      7  59176    Argañaraz              Facundo Nahuel
7           2      8  58909    Argañaraz             Leonardo Ramiro
38          5      2  58876         Díaz              Manuel Lautaro
37  

/var/folders/6n/z412hmln0wn4g5vby875r5s00000gn/T/ipykernel_25899/3696820932.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nombres_duplicados.sort_values(by='nombre', inplace=True)
/var/folders/6n/z412hmln0wn4g5vby875r5s00000gn/T/ipykernel_25899/3696820932.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apellidos_duplicados.sort_values(by='apellido', inplace=True)


# Generar trabajo practicos
Copia de la carpeta practivos el tp a cada una de los repositorios 


In [4]:
# Librería para trabajar con archivos
# def asegurar_directorio(directorio):
#     try:
#         os.makedirs(directorio)
#     except :
#         pass 

def copiar_archivos(origen, destino, forzar=False):
    # Verificar si la carpeta de destino existe, si no, crearla
    if not os.path.exists(destino):
        os.makedirs(destino)
    
    # Iterar sobre todos los archivos en la carpeta de origen
    for archivo in os.listdir(origen):
        ruta_origen  = os.path.join(origen, archivo)
        ruta_destino = os.path.join(destino, archivo)
        
        # Verificar si es un archivo y no un directorio
        if os.path.isfile(ruta_origen):
            if not os.path.exists(ruta_destino) or forzar:
                shutil.copy(ruta_origen, ruta_destino)

def renombrar_carpeta(origen, destino):
    # Crear el directorio de destino si no existe
    if not os.path.exists(destino):
        os.makedirs(destino)

    # Mover el contenido del directorio
    for archivo in os.listdir(origen):
        ruta_origen  = os.path.join(origen, archivo)
        ruta_destino = os.path.join(destino, archivo)

        if os.path.exists(ruta_destino):
            if os.path.isdir(ruta_destino):
                shutil.rmtree(ruta_destino)
            else:
                os.remove(ruta_destino)
        shutil.move(ruta_origen, ruta_destino)
    
    # Eliminar el directorio fuente si está vacío
    if not os.listdir(origen):
        os.rmdir(origen)

In [78]:
def ubicar(comision, *camino):
    if isinstance(comision, int):
        comision = f"/Users/adibattista/Documents/GitHub/Lab4-C{comision}"
    return '/'.join([comision, *camino])

def carpeta_alumno(alumno):
    return f"{alumno['orden']:02} - {alumno['legajo']} - {alumno['apellido']}, {alumno['nombre']}".strip()

def ubicar_alummo(alumnos, legajo):
    alumno = alumnos.loc[alumnos['legajo'] == legajo].iloc[0]
    comision = int(alumno['comision'])
    return ubicar(comision, 'practicos', carpeta_alumno(alumno)).strip()

def publicar(practico):
    alumnos = cargar_alumos()

    # Copia los archivos del practico a cada alumno
    for legajo in alumnos['legajo']:
        destino = f"{ubicar_alummo(alumnos, legajo)}/{practico}" 
        copiar_archivos(f'./practicos/{practico}', destino)
        
    # Copia el enunciados del practico a cada comisión
    for comision in [2,5,7,9]:
        destino = ubicar(comision)
        shutil.copy(f'./practicos/enunciados/{practico}.md', f"{destino}/README.md")

publicar('tp2')

/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/01 - 58764 - Acuña, Ana Sofía
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/02 - 59277 - Aguirre, Emanuel
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/03 - 58827 - Albornoz Silva, Alejo Miguel
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/04 - 58952 - Almiron, Maicol Leonel
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/05 - 59731 - Alvarez, Nicolás
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/06 - 58828 - Alvarez, Nicolás Nahuel
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/07 - 59176 - Argañaraz, Facundo Nahuel
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/08 - 58909 - Argañaraz, Leonardo Ramiro
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/09 - 50665 - Arias Olaiz, Marcos Ignacio
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/10 - 59078 - Bazan, Bruno Gabriel
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/11 - 49185 - Bett, Matías Alejandro
/Users/adib

# Normalizar los nombres de carpetas.
Esta funcion permite revisar que los nombres esten bien escritos (tomando en cuenta el numero de legago)

In [77]:

def normalizar():
    alumnos = cargar_alumos()
    for comision in [2,5,7,9]:
        # listar todas las carpetas de alumnos que esten en la carpeta de la comisión
        carpetas = os.listdir(ubicar(comision, 'practicos'))
        for alumno in carpetas:
            m = re.match(r'\D(\d{5})\D', alumno) 
            if m:
                legajo = m.group(1)
                destino = carpeta_alumno(alumnos.loc[alumnos['legajo'] == legajo].iloc[0])
                alumno = alumno.strip()
                destino = destino.strip()
                if alumno != destino:
                    print(f"in  [{alumno}]\nout [{destino}]\n")
                    src_path  = ubicar(comision, 'practicos', alumno)
                    dest_path = ubicar(comision, 'practicos', destino)
                    # renombrar_carpeta(src_path, dest_path)
                
normalizar()

/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/25 - 58777 - Jassán, Alan/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/24 - 55940 - Gramajo, Elba Virginia Maria/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/05 - 59731 - Alvarez, Nicolás/ejercicio.ipynb
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/14 - 59234 - Chocobar, Matías Sebastián/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/01 - 58764 - Acuña, Ana Sofía/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/29 - 58907 - Nuñez, Walter Exequiel/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/21 - 55533 - González, Luciano Leandro/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/12 - 47121 - Caram, Jesús Nicolás/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/20 - 58832 - González, Adrián Leonel/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/34 - 59424 - Selis, Daiana Betsabé/tp1
/Users/adibattista/Documents/GitHub/Lab4-C2/practicos/15 - 

# Soluciona el proble que publique TP1 directamente en la raiz

In [ ]:
def mover_ejercio_a_tp1():
    alumnos = cargar_alumos()
    for comision in [2,5,7,9]:
        # listar todas las carpetas de alumnos que esten en la carpeta de la comisión
        carpetas = os.listdir(ubicar(comision, 'practicos'))
        for alumno in carpetas:
            alumno = alumno.strip()
            m = re.match(r'\d{2}\s+-?\s*(\d+) - ([^,]+), (.+)', alumno)
            if m:
                legajo = m.group(1)
                origen = ubicar_alummo(alumnos, legajo)
                # alumno = alumnos.loc[alumnos['legajo'] == legajo].iloc[0]

                # origen  = carpeta_alumno(alumno)
                # destino = carpeta_alumno(alumno)+"/tp1"

                # print(f"<{origen}>")
                archivos = os.listdir(origen)
                if '.DS_Store' in archivos: archivos.remove('.DS_Store')

                archivos.sort()
                for archivo in archivos:
                    print(f"{origen}/{archivo}")
                    src_path = f"{origen}/{archivo}"
                    dest_path = f"{origen}/tp1/{archivo}"
                    if not os.path.exists(f"{origen}/tp1"):
                        os.makedirs(f"{origen}/tp1")
                        shutil.move(src_path, dest_path)


                
mover_ejercio_a_tp1()